In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import json
from tensorflow.python.keras.models import Model, load_model
from PIL import Image
PATH = "../tubs/tub6_more_data" #Shadow

C:\Users\allace\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#make sure tensorflow version is the same as one installed on donkeycar (1.8.0)
print(tf.__version__)

1.8.0


In [3]:
from tensorflow.python.client import device_lib
import numpy as np

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

[]

In [4]:
#create df from path
def df_from_jsons(PATH, max_items=None):
    
    #Get json files from path
    json_files = np.array([file for file in os.listdir(PATH) if file.endswith('json') ])
    json_files = json_files[1:]
    np.random.shuffle(json_files)
    
    df = pd.DataFrame()
    for i in json_files[1:max_items+1]:
        with open(f'{PATH}/{i}') as f:
            data = json.load(f)

            userThrottle = data["user/throttle"]
            userAngle = data["user/angle"]
            camImage_path = data["cam/image_array"]
            index = int(camImage_path[:-21])
            camImage_path =  PATH + "/" + camImage_path
            df_row = pd.DataFrame([[userThrottle, userAngle, camImage_path]], index = [index], 
                                  columns = ["user/throttle", "user/angle", "cam/image_array"])

            df = df.append(df_row)
    df = df.sort_index()
    print("Loaded {} json files".format(len(json_files[1:max_items])))
    return df

In [5]:
df_test = df_from_jsons(PATH, max_items=8000)

KeyboardInterrupt: 

In [52]:
df_test.shape

(7999, 3)

In [54]:
df_test = df_test.reset_index()
df_test.head()

,user/throttle,user/angle,cam/image_array
9,0.44,0.01,../tubs/tub6_more_data/9_cam-image_array_.jpg
20,0.34,0.01,../tubs/tub6_more_data/20_cam-image_array_.jpg
26,0.06,-0.17,../tubs/tub6_more_data/26_cam-image_array_.jpg
28,0.08,-0.20,../tubs/tub6_more_data/28_cam-image_array_.jpg
31,0.12,-0.25,../tubs/tub6_more_data/31_cam-image_array_.jpg


In [59]:
df_test.to_feather('tmp/df_test')

In [7]:
df_test = pd.read_feather('tmp/df_test')

C:\Users\allace\Anaconda3\lib\site-packages\pandas\io\feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [8]:
df_test.shape

(7999, 4)

In [9]:
df_test.head()

,index,user/throttle,user/angle,cam/image_array
0,9,0.44,0.01,../tubs/tub6_more_data/9_cam-image_array_.jpg
1,20,0.34,0.01,../tubs/tub6_more_data/20_cam-image_array_.jpg
2,26,0.06,-0.17,../tubs/tub6_more_data/26_cam-image_array_.jpg
3,28,0.08,-0.20,../tubs/tub6_more_data/28_cam-image_array_.jpg
4,31,0.12,-0.25,../tubs/tub6_more_data/31_cam-image_array_.jpg


In [10]:
model = load_model("./models/mypilot5")

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_in (InputLayer)             (None, 120, 160, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 78, 24)   1824        img_in[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 27, 37, 32)   19232       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 12, 17, 64)   51264       conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

In [12]:
def read_record(record_dict, path = PATH):
    data = {}
    for key, val in record_dict.items():
        if key == 'cam/image_array':
            img = Image.open(f'{val}')
            val = np.array(img)
        data[key] = val
    return data

In [13]:
def get_record_gen(record_transform=None, shuffle=True, df=None):
    while True:
        for _ in df.iterrows():
            if shuffle:
                record_dict = df.sample(n=1).to_dict(orient='record')[0]
            
            record_dict = read_record(record_dict)
            
            if record_transform:
                record_dict = record_transform(record_dict)
            
            yield record_dict

In [14]:
def get_batch_gen(keys=None, batch_size=128, record_transform=None, shuffle=True, df=None):
    
    record_gen = get_record_gen(record_transform=record_transform, shuffle=shuffle, df=df)
    
    while True:
        record_list = [next(record_gen) for _ in range(batch_size)]
        
        batch_arrays = {}
        for i, k in enumerate(keys):
            arr = np.array([r[k] for r in record_list])
            batch_arrays[k] = arr
        yield batch_arrays

In [15]:
def get_train_gen(X_keys, Y_keys, batch_size=128, record_transform=None, df=None): #
    batch_gen = get_batch_gen(X_keys + Y_keys, batch_size=batch_size,
                                  record_transform=record_transform, df=df)
    while True:
        batch = next(batch_gen)
        X = [batch[k] for k in X_keys]
        Y = [batch[k] for k in Y_keys] 
        yield X, Y

In [16]:
def get_train_val_gen(df, X_keys, Y_keys, batch_size = 128, train_frac=0.8, #
                      train_record_transform=None, val_record_transform=None): #
    train_df = df.sample(frac=train_frac, random_state=200) #
    val_df = df.drop(train_df.index) #
    
    train_gen = get_train_gen(X_keys=X_keys, Y_keys=Y_keys, batch_size=batch_size,
                             record_transform=train_record_transform, df=train_df)
    val_gen = get_train_gen(X_keys=X_keys, Y_keys=Y_keys, batch_size=batch_size,
                           record_transform=val_record_transform, df=val_df)
    return train_gen, val_gen

In [17]:
def record_transform(record):
    record['user/angle'] = to_bin(record['user/angle'])
    return record

In [18]:
X_keys = ['cam/image_array']
y_keys = ['user/angle', 'user/throttle']
bs = 128 # or 128/3
train_frac = 1
new_model_path = 'models/pilot_jp_2'

In [19]:
def to_bin(a):
    arr = np.zeros(15)
    a = a + 1
    b = round(a/(2/14))
    arr[int(b)] = 1
    return arr

In [20]:
test_gen, _ = get_train_val_gen(df_test, X_keys, y_keys,
                                               train_record_transform=record_transform, #
                                               val_record_transform=record_transform, #
                                               batch_size=bs, #
                                               train_frac=train_frac) #

In [ ]:
%timeit model.evaluate_generator(test_gen, steps=10)

In [ ]:
"""
pilot_BasicReduced: 2.41 s ± 109 ms
mypilot5: 2.63 s ± 146 ms


"""